# sveltejs/svelte

> 

In [1]:
#| default_exp sveltejs-svelte

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import llm
from llm.models import Usage
import anthropic
from pydantic import BaseModel, Field
from jinja2 import Environment, FileSystemLoader
import asyncio
import json
import shutil
from pathlib import Path
import langsmith as ls
import logging

In [4]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [5]:
#| export

source = settings.sources["sveltejs/svelte"]
assert isinstance(source, GitSource)

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

#### Read the markdown files

In [6]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)
# tree.model_dump()

#### Process the tree

In [7]:
# tree.subdirs = tree.subdirs[:2] # For testing process just the first 2 secstion (intro and runes)

In [8]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.394s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 01-introduction/01-overview.md
+0.291s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 01-introduction/02-getting-started.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 01-introduction/03-svelte-files.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 01-introduction/04-svelte-js-files.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 01-introduction/index.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 02-runes/01-what-are-runes.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 02-runes/02-$state.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 02-runes/03-$derived.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 02-runes/04-$effect.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 02-runes/05-$props.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_pa

#### Calculate cost

In [9]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 316,731
  Output tokens: 86,164
  Total tokens: 402,895
Cost:
  Total: $0.75
  Input: $0.32
  Output: $0.43


#### save the results

In [10]:
save_processed_documents(source, settings.output_dir / source.name, processed_tree)

+24.481s DEBUG lovely_docs/docs.py:376 file_map Added . ->  (.)
+0.001s DEBUG lovely_docs/docs.py:376 file_map Added introduction -> introduction (01-introduction)
+0.000s DEBUG lovely_docs/docs.py:384 file_map Added page introduction/overview -> overview (01-introduction/01-overview.md)
+0.000s DEBUG lovely_docs/docs.py:384 file_map Added page introduction/getting-started -> getting-started (01-introduction/02-getting-started.md)
+0.000s DEBUG lovely_docs/docs.py:384 file_map Added page introduction/svelte-files -> svelte-files (01-introduction/03-svelte-files.md)
+0.000s DEBUG lovely_docs/docs.py:384 file_map Added page introduction/.svelte.js-and-.svelte.ts-files -> .svelte.js-and-.svelte.ts-files (01-introduction/04-svelte-js-files.md)
+0.000s DEBUG lovely_docs/docs.py:384 file_map Added page introduction/introduction -> introduction (01-introduction/index.md)
+0.000s DEBUG lovely_docs/docs.py:376 file_map Added runes -> runes (02-runes)
+0.000s DEBUG lovely_docs/docs.py:384 file_m